<a href="https://colab.research.google.com/github/bharagvrathod/negotiation_chatbot/blob/main/negotiation_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Negotiation chatbot**

This project features two negotiation chatbot models using the DialoGPT model from Hugging Face. Both chatbots engage in price negotiation with users, offering and adjusting prices based on user input. The second model enhances this by incorporating sentiment analysis, providing context-aware, sentiment-driven responses. This chatbot can be used to simulate price negotiations with AI that adapts to user interaction.

In [2]:
!pip install transformers torch


# **Negotiation Chatbot Using DialoGPT without sentiment analysis**

## 1. **Basic Negotiation Model (Non-Sentiment)**


* ### DialoGPT Integration:
    * The chatbot uses the DialoGPT-medium model for generating natural language responses. The model generates dialogue based on the user’s input and previous conversation.  
* ### Price Negotiation Logic
    * **Initial Offer**: The bot starts with a maximum offer, pre-set at $120.
    * **Counteroffer Handling**: Users can either accept the current offer or propose a counteroffer.
    * **Price adjustment**: If the user's counteroffer is close to the last offer (within $5), the bot accepts the user’s price. If the counteroffer is too low (below $80), the bot rejects it and asks for a higher offer. If the counteroffer is acceptable but not close enough to finalize the deal, the bot makes a new counteroffer as the average between the previous offer and the user’s price.




In [40]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the DialoGPT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Set acceptable price range for negotiation
BASE_PRICE = 100
MIN_PRICE = 80
MAX_PRICE = 120

previous_offers = {
    'chatgpt_offer': MAX_PRICE  # Initial offer is set to MAX_PRICE
}

# Function to get DialoGPT response
def get_dialo_response(user_input):
    # Encode the user input and previous conversation
    input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Generate a response
    chat_history_ids = model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # Decode the response
    response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response

# Function to initiate the negotiation
def initiate_negotiation():
    prompt = (f"Initiate a negotiation for a product price. "
              f"Offer a reasonable starting price between {MIN_PRICE} and {MAX_PRICE}, "
              f"and encourage the user to either accept, reject, or propose a counteroffer.")

    initial_offer = f"The initial offer is ${MAX_PRICE}. What do you think?"
    previous_offers['chatgpt_offer'] = MAX_PRICE
    return initial_offer

# Function to handle user counteroffers
def negotiate(user_price, agreed=False):
    last_offer = previous_offers['chatgpt_offer']

    if agreed:
        return f"Thank you for agreeing to ${user_price}. The deal is finalized!"

    if abs(last_offer - user_price) <= 5:
        return f"Accepted your offer of ${user_price}. The deal is finalized!"

    if user_price < MIN_PRICE:
        return f"Your offer of ${user_price} is below the minimum. Please offer more."

    if user_price >= MIN_PRICE:
        new_offer = (user_price + last_offer) // 2  # Propose a new counteroffer
        previous_offers['chatgpt_offer'] = new_offer
        return f"I propose a new offer of ${new_offer}."

# Start the negotiation
initial_response = initiate_negotiation()
print("ChatGPT's Initial Offer:", initial_response)

# Simulate user interactions
while True:
    user_input = input("Enter your price (or type 'accept' to agree): ")
    if user_input.lower() == 'accept':
        final_response = negotiate(user_price, agreed=True)
        print(final_response)
        break
    else:
        try:
            user_price = int(user_input)
            response = negotiate(user_price)
            print("ChatGPT's Response:", response)
        except ValueError:
            print("Please enter a valid price or 'accept' to agree.")


ChatGPT's Initial Offer: The initial offer is $120. What do you think?
Enter your price (or type 'accept' to agree): 40
ChatGPT's Response: Your offer of $40 is below the minimum. Please offer more.
Enter your price (or type 'accept' to agree): 80
ChatGPT's Response: I propose a new offer of $100.
Enter your price (or type 'accept' to agree): 95
ChatGPT's Response: Accepted your offer of $95. The deal is finalized!
Enter your price (or type 'accept' to agree): accept
Thank you for agreeing to $95. The deal is finalized!


# **Negotiation Chatbot Using DialoGPT with sentiment analysis**

* **DialoGPT Integration**:
    * Like the first model, the second chatbot uses the DialoGPT-medium model for generating responses.
* **Sentiment Analysis Integration**:
    * This version incorporates Hugging Face's sentiment analysis pipeline. The chatbot evaluates whether user input is positive, neutral, or negative and adjusts its responses accordingly.
* **Price Negotiation Logic with Sentiment**:
    * **Initial Offer**: The bot initiates the negotiation with a maximum offer of $120.
        * **Sentiment-Based Price Adjustments**:
            * **Positive Sentiment**: If the user's sentiment is positive, the bot is more generous, reducing the offer by up to 20%.
            * **Neutral Sentiment**: For neutral responses, the bot makes smaller adjustments, typically reducing the offer by around 7%.
            * **Negative Sentiment**: If the user shows negative sentiment, the bot may slightly increase the price (around 3%) while responding more carefully.
            * **Response Generation**: The bot uses polite and sometimes humorous responses to guide the conversation, especially when a positive sentiment is detected.

In [41]:
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Load the DialoGPT model and tokenizer from Hugging Face's model hub
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Fix the pad_token issue: If the tokenizer doesn't have a pad token set, use the end-of-sequence token instead.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load a sentiment analysis pipeline to evaluate user input sentiment
sentiment_analyzer = pipeline("sentiment-analysis")

# Set acceptable price range for negotiation
BASE_PRICE = 100  # Base price for negotiations
MIN_PRICE = 80    # Minimum acceptable price
MAX_PRICE = 120   # Maximum acceptable price

# Dictionary to store previous offers made by ChatGPT
previous_offers = {
    'chatgpt_offer': MAX_PRICE  # Initial offer is set to MAX_PRICE
}

# Function to get a humorous and polite GPT response based on the new offer and user sentiment
def get_dialo_response(new_offer, sentiment_label, user_price):
    humor_prompt = ""

    # Determine how to respond based on sentiment
    if sentiment_label == 'POSITIVE':
        humor_prompt = "The user seems polite and positive. Please offer a humorous and generous response, appreciating their attitude."
    elif sentiment_label == 'NEGATIVE':
        humor_prompt = "The user seems unhappy. Please respond kindly but firmly, explaining the reasons for the current offer."
    else:
        humor_prompt = "The user seems neutral. Please offer a polite and slightly humorous response, sticking to a reasonable counteroffer."

    # Create a prompt for GPT with the current offer, sentiment, and the user's offer
    prompt = (
        f"The user has offered ${user_price}, and the counteroffer is now ${new_offer}. "
        f"{humor_prompt} Keep the tone professional but humorous."
    )

    # Encode the prompt into input IDs for the model
    input_ids = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors='pt')

    # Generate a response from the model
    chat_history_ids = model.generate(
        input_ids,
        max_length=500,
        pad_token_id=tokenizer.pad_token_id,
        attention_mask=torch.ones_like(input_ids),
        do_sample=True
    )

    # Decode the generated response back into text
    response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response

# Function to initiate the negotiation process by making the initial offer
def initiate_negotiation():
    initial_offer = f"The initial offer is ${MAX_PRICE}. What do you think?"  # Create initial offer message
    previous_offers['chatgpt_offer'] = MAX_PRICE  # Update the previous offer
    return initial_offer

# Function to adjust the offer based on user sentiment
def adjust_offer_based_on_sentiment(sentiment_label, last_offer):
    if sentiment_label == 'POSITIVE':
        # For positive sentiment, reduce the offer more significantly (e.g., up to 20%)
        reduction = (MAX_PRICE - MIN_PRICE) * 0.2
        new_offer = max(MIN_PRICE, last_offer - reduction)
    elif sentiment_label == 'NEGATIVE':
        # For negative sentiment, slightly increase the offer (up to 3%)
        increase = (MAX_PRICE - MIN_PRICE) * 0.03
        new_offer = min(MAX_PRICE, last_offer + increase)
    else:
        # For neutral sentiment, make a smaller reduction (e.g., 7%)
        adjustment = (MAX_PRICE - MIN_PRICE) * 0.07
        new_offer = max(MIN_PRICE, last_offer - adjustment)

    # Update the previous offer to the new calculated offer
    previous_offers['chatgpt_offer'] = new_offer
    return new_offer

# Function to extract a numeric price from user input
def extract_price(user_input):
    match = re.search(r'\d+', user_input)  # Look for a number in the input
    if match:
        return int(match.group())  # Return the extracted number as an integer
    return None  # Return None if no valid number is found

# Function to handle the negotiation response from the user
def handle_negotiation(user_input):
    last_offer = previous_offers['chatgpt_offer']  # Get the last offer made by ChatGPT

    user_price = extract_price(user_input)  # Attempt to extract a price from user input
    if user_price is None:
        return "Please enter a valid price or 'accept' to agree."  # Handle invalid input

    sentiment_result = sentiment_analyzer(user_input)[0]  # Analyze the sentiment of the user input
    sentiment_label = sentiment_result['label']  # Extract the sentiment label (POSITIVE, NEGATIVE, or NEUTRAL)

    new_offer = adjust_offer_based_on_sentiment(sentiment_label, last_offer)  # Adjust the offer based on sentiment
    response = get_dialo_response(new_offer, sentiment_label, user_price)  # Get the response from GPT based on the new offer

    return f"My counteroffer is ${new_offer}. {response}"  # Return the new offer and response

# Start the negotiation by initiating the first offer
initial_response = initiate_negotiation()
print("ChatGPT's Initial Offer:", initial_response)  # Print the initial offer

# Simulate user interactions in a loop
while True:
    user_input = input("You: ")  # Prompt for user input
    if user_input.lower() == 'accept':  # Check if the user wants to accept the offer
        final_response = f"Thank you for accepting the offer of ${previous_offers['chatgpt_offer']}. The deal is finalized!"  # Finalize the deal
        print(final_response)  # Print the final response
        break  # Exit the loop
    else:
        response = handle_negotiation(user_input)  # Handle negotiation based on user input
        print("ChatGPT:", response)  # Print ChatGPT's response


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ChatGPT's Initial Offer: The initial offer is $120. What do you think?
You: thank you, can we do at 95 please
ChatGPT: My counteroffer is $112.0. Good advice.
You: thank you, 97 would be great please
ChatGPT: My counteroffer is $104.0. Thank you very much. I appreciate the counteroffer, you would not have had it if I didn't offer it.
You: thank you, 98 please
ChatGPT: My counteroffer is $96.0. I'll let both of the users do the negotiating.
You: accept
Thank you for accepting the offer of $96.0. The deal is finalized!
